# Q5

In [37]:
using DelimitedFiles, Convex, SCS

In [38]:
wine_data = readdlm("winequality-red.csv", ';', Float64, skipstart=1);

In [39]:
Y = wine_data[:, 12]
X = wine_data[:, 1:11];

In [40]:
train_X = X[1:1400, :]
train_Y = Y[1:1400, :]
test_X = X[1401:1599, :]
test_Y = Y[1401:1599, :]
train_X = [train_X ones(1400, 1)]
test_X = [test_X ones(199, 1)]
weight = train_X\train_Y

12×1 Matrix{Float64}:
   0.02329876002903944
  -1.0762050522637037
  -0.1452817271412922
   0.006569057673688157
  -1.8254194334176277
   0.0034154908282525913
  -0.0034275751572619166
 -14.95044577183303
  -0.29189594818793224
   0.873197346652471
   0.27765633249787985
  18.70970670509345

In [41]:
least_square_loss = (sum(abs.(test_X * weight - test_Y)))/199
β_huber = Variable(12)
cost = sum(huber(train_X * β_huber - train_Y));
solve!(minimize(cost), ()->SCS.Optimizer(verbose=0))
hub_loss = (sum(abs.(test_X * β_huber.value - test_Y)))/199
hinge(x) = max(abs(x)-0.5, 0)
β_hinge = Variable(12)
cost = sum(hinge(train_X * β_hinge - train_Y));
solve!(minimize(cost), ()->SCS.Optimizer(verbose=0))
hinge_loss = (sum(abs.(test_X * β_hinge.value - test_Y)))/199

┌ Warning: Problem status ALMOST_OPTIMAL; solution may be inaccurate.
└ @ Convex C:\Users\Malvika\.julia\packages\Convex\uI27T\src\solution.jl:263


0.5482098002865922

In [42]:
print("For least square loss Mean Absolution Error(MAE) is ", least_square_loss, "\n")
print("______________________________________________________________________________\n")
print("For huber loss Mean Absolution Error(MAE) is ", hub_loss, "\n")
print("______________________________________________________________________________\n")
print("For hinge loss Mean Absolution Error(MAE) is ", hinge_loss, "\n")
print("______________________________________________________________________________\n")

For least square loss Mean Absolution Error(MAE) is 0.5329671066366277
______________________________________________________________________________
For huber loss Mean Absolution Error(MAE) is 0.5327150885601937
______________________________________________________________________________
For hinge loss Mean Absolution Error(MAE) is 0.5482098002865922
______________________________________________________________________________


# Q6

In [43]:
using CSV, DataFrames, Convex, SCS
iono_data = DataFrame(CSV.File("ionosphere.data";header=0))
Y = ones(351,1)
for i in 1:351
    if iono_data[i,35]=="g"
        Y[i] = 1
    else
        Y[i] = -1
    end
end
iono_data = Matrix(iono_data)
X = iono_data[:,1:34]
X = convert(Array{Float64,2}, X)

351×34 Matrix{Float64}:
 1.0  0.0   0.99539  -0.05889   0.85243  …  -0.54487   0.18641  -0.453
 1.0  0.0   1.0      -0.18829   0.93035     -0.06288  -0.13738  -0.02447
 1.0  0.0   1.0      -0.03365   1.0         -0.2418    0.56045  -0.38238
 1.0  0.0   1.0      -0.45161   1.0          1.0      -0.32382   1.0
 1.0  0.0   1.0      -0.02401   0.9414      -0.59573  -0.04608  -0.65697
 1.0  0.0   0.02337  -0.00592  -0.09924  …   0.0      -0.00039   0.12011
 1.0  0.0   0.97588  -0.10602   0.94601     -0.81318  -0.13832  -0.80975
 0.0  0.0   0.0       0.0       0.0          1.0       0.0       0.0
 1.0  0.0   0.96355  -0.07198   1.0         -0.72779   0.38895  -0.7342
 1.0  0.0  -0.01864  -0.08459   0.0          0.24086  -0.08208   0.38065
 1.0  0.0   1.0       0.06655   1.0      …  -0.89093   0.22995  -0.89158
 1.0  0.0   1.0      -0.5421    1.0         -1.0       1.0      -1.0
 1.0  0.0   1.0      -0.16316   1.0         -0.64537   0.64727  -0.67226
 ⋮                                       ⋱

In [44]:
train_x = X[1:300,:]
train_y = Y[1:300,:]
test_x = X[301:351,:]
test_y = Y[301:351,:]
train_x = [train_x ones(300,1)]
test_x = [test_x ones(51,1)]
ω = train_x\train_y
actual = 0
for i in 1:51
    predicted = dot(test_x[i,:],ω)
    if predicted > 0 && test_y[i]>0
        actual = actual + 1
    end
    if predicted < 0 && test_y[i,:]<0
        actual = actual + 1
    end
end
print("Least square loss \n-----------------------------\nPrediction accuracy: ",(actual/51)*100,"% \n")

Least square loss 
-----------------------------
Prediction accuracy: 100.0% 


In [45]:
β_logistic = Variable(35)
cost = sum(logisticloss(-train_y.*(train_x*β_logistic)))
solve!(minimize(cost), ()-> SCS.Optimizer(verbose=0))
actual = 0
for i in 1:51
    predicted = dot(test_x[i,:],β_logistic.value)
    if predicted > 0 && test_y[i]>0
        actual = actual + 1
    end
    if predicted < 0 && test_y[i,:]<0
        actual = actual + 1
    end
end
print("Logistic loss \n-----------------------------\nPrediction accuracy: ",(actual/51)*100,"% \n")

Logistic loss 
-----------------------------
Prediction accuracy: 100.0% 
WARN: A->p (column pointers) not strictly increasing, column 1 empty


In [46]:
hinge(y,t) = max(1-y.*t,0)
beta_hinge = Variable(35)
cost = sum(hinge(train_y,train_x*beta_hinge))
solve!(minimize(cost), ()-> SCS.Optimizer(verbose=0))
actual = 0
for i in 1:51
    predicted = dot(test_x[i,:],beta_hinge.value)
    if predicted > 0 && test_y[i]>0
        actual = actual + 1
    end
    if predicted < 0 && test_y[i,:]<0
        actual = actual + 1
    end
end
print("Hinge Loss \n-----------------------------\nPrediction accuracy: ",(actual/51)*100,"% \n")

Hinge Loss 
-----------------------------
Prediction accuracy: 100.0% 
WARN: A->p (column pointers) not strictly increasing, column 301 empty
